In [ ]:
import torch
import nltk
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader, random_split, dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'nltk'

In [ ]:
Embed_dim = 300
word_num = 10_000

In [ ]:
class Rnn(nn.Module):
    def __init__(self):
        super(Rnn, self).__init__()
        self.embedding = nn.Embedding(embedding_dim=Embed_dim)
        self.rnnL1 = nn.RNN(input_size=Embed_dim ,hidden_size=1024,nonlinearity= nn.Tanh())
        self.linL2 = nn.Linear(1024,5)
        # Wrong, you need to classify a number with the review text
        
    def forward(x,self):
        x=self.rnnL1(x)
        x=self.linL2(x)
        x=nn.Softmax(x)
        return(x)
        


Dataset call section

In [ ]:
dir_path = r"C:\Users\orian\OneDrive\שולחן העבודה\Coding Files 2\Datasets"
dataset = pd.read_csv(dir_path+r"\chat gpt reviews\ChatGPT_Reviews.csv")
emb_dict = pd.read_csv(dir_path+r"\embedding dictionary\dict(2).csv")

print(emb_dict.keys())


part_ds = [dataset['Review'], dataset['Ratings']]
train_len = round(len(part_ds)*0.9)
train_ds_csv, test_ds_csv = random_split(
    dataset=part_ds,lengths=[train_len, len(part_ds)-train_len])
train_ds = DataLoader(train_ds_csv, shuffle= True)

Embedding Section

In [ ]:
import torchgen.gen


word_dict = emb_dict['word']
word_dict = tuple(set(word_dict))
print(len(word_dict))
word_to_index = {key: val for val, key in enumerate(word_dict)}  # Dictionary to map words to indices
embedding_layer = nn.Embedding(len(word_to_index), Embed_dim)

input_words = ["suck", "the","shit"]
input_indices = torch.LongTensor([word_to_index[word] for word in input_words])  # Convert words to indices

embeddings = embedding_layer(input_indices)

print(torchgen)
print(embeddings)